In [46]:
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.optim as optim

In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import pickle
warnings.filterwarnings('ignore')

In [48]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder

In [49]:
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [50]:
data = data.drop(columns=['RowNumber','CustomerId','Surname'],axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [51]:
label_encoder = LabelEncoder()
data['Gender'] = label_encoder.fit_transform(data['Gender'])
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [52]:
one_hot = OneHotEncoder()
encoded_data = one_hot.fit_transform(data[['Geography']])
encoded_data

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [53]:
encoded_data = encoded_data.toarray()
encoded_data

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [54]:
one_hot.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [55]:
encoded_data_geography = pd.DataFrame(encoded_data,columns=one_hot.get_feature_names_out(['Geography']))
encoded_data_geography.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [56]:
data = pd.concat([data.drop('Geography',axis=1),encoded_data_geography],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [57]:
with open('label_encoder.pkl','wb') as file:
  pickle.dump(label_encoder,file)

with open('one_hot_encoder.pkl','wb') as file:
  pickle.dump(one_hot,file)

In [58]:
X = data.drop('Exited',axis=1)
y = data['Exited']

In [59]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [60]:
with open('scaler.pkl','wb') as file:
  pickle.dump(sc,file)

In [61]:
import torch
class Churn_Modelling(nn.Module):
    def __init__(Churn_Modelling,self):
        super(Churn_Modelling,self).__init__()
        self.cf1 = nn.Linear(X.shape[1],64) # input layer X.shape[1] = 12 that is input features
        self.cf2 = nn.Linear(64,32) # hidden layer
        self.output = nn.Linear(32,1) # output layer

    def forward(self,x):
        x = F.relu(self.cf1(x))
        x = F.relu(self.cf2(x))
        x = torch.sigmoid(self.output(x))
        return x

In [62]:
model = Churn_Modelling()
criterion = nn.BCELoss() # binary cross entropy loss
optimizer = optim.Adam(model.parameters(),lr=0.01)

epochs = 100
for epoch in range(epochs):
    inputs = torch.tensor(X_train,dtype=torch.float32)
    labels = torch.tensor(y_train,dtype=torch.float32)

    outputs = model(inputs)
    loss = criterion(outputs,labels.unsqueeze(1))

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss {loss.item():.4f}')

Epoch [10/100], Loss 0.4612
Epoch [20/100], Loss 0.4272
Epoch [30/100], Loss 0.4142
Epoch [40/100], Loss 0.4000
Epoch [50/100], Loss 0.3815
Epoch [60/100], Loss 0.3681
Epoch [70/100], Loss 0.3538
Epoch [80/100], Loss 0.3310
Epoch [90/100], Loss 0.3201
Epoch [100/100], Loss 0.3112


In [63]:
with torch.no_grad():
    y_pred = model(torch.tensor(X_test,dtype=torch.float32))
    y_pred = y_pred.round()
    acc = y_pred.eq(torch.tensor(y_test.values,dtype=torch.float32).unsqueeze(1)).sum() / float(y_test.size)
    print(f'Accuracy: {acc:.4f}')

Accuracy: 0.8555


In [64]:
torch.save(model.state_dict(),'model.pth')

In [65]:
torch.save(model,'model.h5')

In [66]:
new = torch.load('model.h5')
new.eval()

Churn_Modelling(
  (cf1): Linear(in_features=12, out_features=64, bias=True)
  (cf2): Linear(in_features=64, out_features=32, bias=True)
  (output): Linear(in_features=32, out_features=1, bias=True)
)

In [67]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cuda device
